In [ ]:
import datetime
import glob
import os

import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('nbs.mplstyle')
%matplotlib inline
import numpy as np
import pandas as pd

from scipy.stats import binned_statistic
from scipy.ndimage import gaussian_filter1d

os.makedirs('../plots', exist_ok=True)

In [ ]:
dfs = {}

for filename in glob.glob('../cache/*fulltext.csv'):
    df = pd.read_csv(filename, index_col='pubdate', parse_dates=True)
    name = os.path.basename(filename).split('-fulltext')[0]
    dfs[name] = df

In [ ]:
for yscale in ['linear', 'log']:
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))

    for name in sorted(dfs.keys()):
        df = dfs[name]

        g = df.groupby(by=[df.index.year]).count()
        group_dates = np.array([datetime.date(x, 1, 1) for x in g.index])
        mentions = g['pubdate.1'].values

        if name == 'julia':
            jmask = group_dates > datetime.date(2012, 1, 1)
            group_dates = group_dates[jmask]
            mentions = mentions[jmask]

        ax.plot(group_dates, 
                mentions, 
                marker='', drawstyle='steps-mid', 
                lw=2, label=name)

    ax.set_xlim(datetime.date(1991, 8, 1),
                datetime.datetime.now() - datetime.timedelta(days=60))
    
    ax.legend(loc='upper left', fontsize=16)

    ax.set_xlabel('time')
    ax.set_ylabel('Fulltext mentions per year')

    ax.set_yscale(yscale)

    fig.tight_layout()
    
    fig.savefig(f'../plots/prog-lang-fulltext-{yscale}.png', dpi=300)

In [ ]:
for yscale in ['linear', 'log']:
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))

    for name in sorted(dfs.keys()):
        df = dfs[name]

        g = df.groupby(by=[df.index.year, df.index.month]).count()
        
        group_dates = np.array([datetime.date(x[0], x[1], 1) for x in g.index])
        mentions = g['pubdate.1'].values

        if name == 'julia':
            jmask = group_dates > datetime.date(2012, 1, 1)
            group_dates = group_dates[jmask]
            mentions = mentions[jmask]

        ax.plot(group_dates, 
                mentions, 
                marker='', drawstyle='steps-mid', 
                lw=1, label=name)

    ax.set_xlim(datetime.date(1991, 8, 1),
                datetime.datetime.now() - datetime.timedelta(days=60))
    
    ax.legend(loc='upper left', fontsize=16)

    ax.set_xlabel('time')
    ax.set_ylabel('Fulltext mentions per month')

    ax.set_yscale(yscale)

    fig.tight_layout()
    
    fig.savefig(f'../plots/prog-lang-fulltext-{yscale}-monthly.png', dpi=300)